In [1]:
%cd ..

/home/ec2-user/uclresearchanalysis


In [2]:
import builtins
# builtins.uclresearch_topic = 'GIVENCHY'
builtins.uclresearch_topic = 'HAWKING'
# builtins.uclresearch_topic = 'NYC'
# builtins.uclresearch_topic = 'FLORIDA'
from configuration import config
from configuration import crawl

('Reading config file from location: '
 '/home/ec2-user/uclresearchanalysis/configuration/env.properties')
{'calculate': {'analysis': True,
               'friends': True,
               'network': True,
               'uniquetweets': True,
               'uniqueusers': True},
 'data': {'dates': ['2018-03-11', '2018-03-12', '2018-03-13'],
          'eventname': "Givenchy's Death",
          'phrases': ['givenchy%20death', 'givenchy%20passed%20away'],
          'starttime': 'Mar 12 08:20:00 -0500 2018'},
 'path': {'crawl': {'followers': '/home/ec2-user/uclresearchanalysis/followers',
                    'friends': '/home/ec2-user/uclresearchanalysis/friends'},
          'cwd': '/home/ec2-user/uclresearchanalysis/data/givenchy',
          'ml': '/home/ec2-user/uclresearchanalysis/data/givenchy/pickle',
          'networkx': {'all': '/home/ec2-user/uclresearchanalysis/data/givenchy/pickle/networkx_all.dat',
                       'friends': '/home/ec2-user/uclresearchanalysis/data/givench

In [3]:
import time
import tweepy
from tweepy import OAuthHandler
import json
import datetime as dt
import os
import sys
import pickle
from tqdm import tqdm
import datetime
import pandas as pd
import threading
import time


In [4]:
need_to_crawl = config.load_followers_needcrawl_set()

Loading data file from path /home/ec2-user/uclresearchanalysis/data/givenchy/pickle/followersneedcrawl.dat
'Loaded 0 entires'


In [ ]:
TwitterApi

In [5]:
class TwitterApi(object):
    def __init__(self, consumer_key, consumer_secret, access_token, access_secret): 
        self.consumer_key = consumer_key 
        self.consumer_secret = consumer_secret 
        self.access_token = access_token
        self.access_secret = access_secret

    def loadapi(self):
        auth = OAuthHandler(self.consumer_key, self.consumer_secret)
        auth.set_access_token(self.access_token, self.access_secret)
        return tweepy.API(
            auth, 
            wait_on_rate_limit=True,
            wait_on_rate_limit_notify=True, 
            compression=True)

In [6]:
def crawl_follower(thread_name, api, user_id):
    follower_ids = []
    try:
        c = tweepy.Cursor(api.followers_ids, id = user_id)
        for page in c.pages():
            follower_ids.extend(page)
    except tweepy.TweepError as e:
        print('tweepy.TweepError', e, ' from: ', thread_name)
    except:
        e = sys.exc_info()[0]
        print("Error: {}, from: {}".format(e, thread_name))
    return follower_ids

In [7]:
def save_followers(thread_name, followers_dictionary):
    filename = 'newfollowers-{date:%Y-%m-%d %H:%M:%S}-{}.dat'.format(
        thread_name, 
        date=datetime.datetime.now())
    config.dump_new_follwers_dictionary(followers_dictionary, filename)

In [8]:
class crwalThread(threading.Thread):
    def __init__(self, thread_id, thread_name, api, user_ids):
        threading.Thread.__init__(self)
        self.thread_id = thread_id
        self.thread_name = thread_name
        self.api = api
        self.user_ids = user_ids
    def run(self):
        print('{}: {}'.format(self.thread_name, ': starting'))
        dump_dictionary_if_over_this_size = 15
        followers_dictionary = {}
        while len(self.user_ids) > 0:
            user_id = self.user_ids.pop()
            followers_dictionary[user_id] = crawl_follower(self.thread_name, self.api, user_id)
            if len(followers_dictionary.keys()) >= dump_dictionary_if_over_this_size:
                save_followers(self.thread_name, followers_dictionary)
                followers_dictionary = {}
        save_followers(self.thread_name, followers_dictionary)
        print('{}: {}'.format(self.thread_name, ': exiting'))

In [9]:
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))

In [10]:
keys = config.load_twitter_keys()
number_of_apis = len(keys.index)
print('Found {} apis, could crawl {} followers per day'.format(keys.shape[0], keys.shape[0] * 60 * 24))
need_to_crawl = list(config.load_followers_needcrawl_set())
print('Number of users we still need to crawl: {}'.format(len(need_to_crawl)))
splitted_user_ids = list(split(need_to_crawl, keys.shape[0]))

threads = []
for index, row in keys.iterrows():    
    twitterApi = TwitterApi(row.consumer_key, row.consumer_secret, row.access_token, row.access_secret).loadapi()
    print('Starting index: {}, name: {}, with {} entries'.format(index, row.reference, len(splitted_user_ids[index])))
    threads.append(crwalThread(index, row.reference, twitterApi, splitted_user_ids[index]))

for each_thread in threads:
    each_thread.start()    
    
for each_thread in threads:
    each_thread.join()
    
print("Finished crawling")

Found 38 apis, could crawl 54720 friendships per day
Loading data file from path /home/ec2-user/uclresearchanalysis/data/givenchy/pickle/followersneedcrawl.dat
'Loaded 0 entires'
Number of users we still need to crawl: 0
Starting index: 0, name: Jimmy_Canteen, with 0 entries
Starting index: 1, name: BarberDdddavid, with 0 entries
Starting index: 2, name: Hu13Steve, with 0 entries
Starting index: 3, name: james13_david, with 0 entries
Starting index: 4, name: hui, with 0 entries
Starting index: 5, name: lzhoudevuk, with 0 entries
Starting index: 6, name: lzhoudevcn, with 0 entries
Starting index: 7, name: lzhoudevau, with 0 entries
Starting index: 8, name: ucablz4, with 0 entries
Starting index: 9, name: liyi.zhou.17, with 0 entries
Starting index: 10, name: lzhou1110, with 0 entries
Starting index: 11, name: wu, with 0 entries
Starting index: 12, name: peter.liyi.z, with 0 entries
Starting index: 13, name: miao, with 0 entries
Starting index: 14, name: adlrl, with 0 entries
Starting in